In [1]:
using Pkg
Pkg.activate("/media/mat/HDD/AdaptiveTransportMap/")

 Activating environment at `/media/mat/HDD/AdaptiveTransportMap/Project.toml`


In [62]:
using Revise
using AdaptiveTransportMap
using LinearAlgebra
import LinearAlgebra: ldiv!, dot
using Test
using ForwardDiff
using SpecialFunctions
using BenchmarkTools
using TransportMap
using QuadGK
using Polynomials
using Distributions
using Random
using LoopVectorization
using Optim
using NLsolve
using MLDataUtils
using MLDataPattern
using Test
using SparseArrays
using AdaptiveTransportMap: vander, transform!, evaluate, ncoeff, optimize, negative_log_likelihood!, derivative

# using Profile
using ProfileView

In [3]:
Threads.nthreads()

8

### Integration

In [85]:
Nx = 20
Ny = 1
m = 20
Ne = 300
X = randn(Nx, Ne).*randn(Nx, Ne) #.+ 0.5*randn(Nx).*cos.(randn(Nx, Ne) .* randn(Nx, Ne))

# L = LinearTransform(X)
# transform!(L, X);
# C = MapComponent(m, Nx)
# @time C, _ = optimize(C, X, 10)

In [86]:
M = HermiteMap(m, X);

In [87]:
@time optimize(M, X, 10; P = serial)

  1.259750 seconds (1.00 M allocations: 1.347 GiB, 6.61% gc time)


HermiteMap(20, 20, LinearTransform(20, [-0.015911955921730166, 0.013971261551210897, -0.040447883932978615, 0.0753439815071193, -0.0781197937126532, -0.02599238710934617, 0.06977623841979197, 0.026324732718774484, -0.053128114005743056, -0.059599905362144906, 0.001114149051942673, 0.04695820217296057, -0.016624913310568697, -0.010312831186233277, -0.07986824550904248, -0.01603764087753745, -0.01717321084123384, -0.05563818622888866, -0.006488511882438942, 0.012474994994471028], [0.8004942544470827 0.0 … 0.0 0.0; 0.0 0.9074870450033097 … 0.0 0.0; … ; 0.0 0.0 … 0.9754789867528696 0.0; 0.0 0.0 … 0.0 1.0062688660692523], true), MapComponent[MapComponent(20, 10, 1, IntegratedFunction(20, 10, 1, Rectifier("softplus"), ParametricFunction(ExpandedFunction(20, 10, 1, MultiBasis(Basis of 20 functions: Constant -> 18th degree Probabilistic Hermite function
, 1), [1; 2; … ; 9; 10], [-0.026624505570497812, -1.323025924661739, 0.3868818352744172, -4.249212928226716, 1.5499142401581412, -2.3980726711

In [28]:
function timing()
    @btime begin 
        M = HermiteMap($m, $X)
        optimize($M, $X, $10; P = serial)
    end
end

timing (generic function with 1 method)

In [29]:
timing();

  973.450 ms (1054425 allocations: 1.43 GiB)


In [30]:
function timing()
    @btime begin 
        M = HermiteMap($m, $X)
        optimize($M, $X, $10; P = thread)
    end
end

timing (generic function with 1 method)

In [88]:
timing();

  297.730 ms (1000725 allocations: 1.35 GiB)


In [118]:
# @testset "Test inversion of the Hermite Map" begin

    Nx = 50
    Ny = 20
    m = 20
    Ne = 500
    Xprior = randn(Nx, Ne).*randn(Nx, Ne)
    Xprior_thread = deepcopy(Xprior)
    Xpost = deepcopy(Xprior) .+ 0.2*randn(Nx, Ne)

    M = HermiteMap(m, Xprior; diag = true)
    Ystar = deepcopy(Xpost[1:Ny,:])# + 0.05*randn(Ny,Ne);

    M = HermiteMap(m, Xprior)
    optimize(M, Xprior, 5; withconstant = false, start = Ny+1)

    F = evaluate(M, Xpost; start = Ny+1)
    inverse!(F, M, Xprior, Ystar; start = Ny+1, P = serial)

    inverse!(F, M, Xprior_thread, Ystar; start = Ny+1, P = thread)

    @test norm(Xprior - Xprior_thread)<1e-8

    @test norm(evaluate(M, Xprior; start = Ny+1)-evaluate(M, Xpost; start = Ny+1))/norm(evaluate(M, Xpost; start = Ny+1))<1e-8

    @test norm(Xprior[Ny+1:end,:] - Xpost[Ny+1:end,:])/norm(Xpost[Ny+1:end,:])<1e-8
# end

  0.000001 seconds (1 allocation: 208 bytes)
Test Failed at In[118]:22
  Expression: norm(Xprior - Xprior_thread) < 1.0e-8
   Evaluated: 22.289195115787102 < 1.0e-8


Test.FallbackTestSetException: There was an error during testing

In [119]:
Xprior

50×500 Array{Float64,2}:
 -0.078502     0.324462    0.936004   …  -0.450302   -2.31333
 -0.171019     1.17809    -0.185392       0.0602692  -1.20097
 -1.29727      0.612046    4.66302       -0.019651   -0.940273
  0.657376     0.125466   -0.629778       1.69212     0.216574
  0.443962    -0.0327877   0.286606       3.48325     1.79368
  4.21507      0.137973   -0.168136   …  -0.392907   -0.0498037
 -1.93143     -0.431774   -0.729711       0.0891769  -2.31571
 -0.275619    -1.97767     0.344649       1.82559     0.581243
 -0.132816     0.0393465   3.82573        0.187089    0.605982
  0.636026     0.215633   -1.53364       -0.043997    0.803863
 -0.454117     1.24044     0.212226   …   0.930079   -0.00264196
 -0.404232     0.400938    0.180427      -0.0598841  -0.015303
  0.104326    -0.0533569  -0.310413       0.368739    0.180631
  ⋮                                   ⋱              
  0.2157       0.976283    1.33829        0.560527    0.270111
  1.70194     -0.215206    0.161297     

In [121]:
Xprior_thread

50×500 Array{Float64,2}:
 -0.078502    0.324462     0.936004   -0.201444   …  -0.450302   -2.31333
 -0.171019    1.17809     -0.185392    0.0164561      0.0602692  -1.20097
 -1.29727     0.612046     4.66302    -0.581911      -0.019651   -0.940273
  0.657376    0.125466    -0.629778   -0.70921        1.69212     0.216574
  0.443962   -0.0327877    0.286606   -0.118804       3.48325     1.79368
  4.21507     0.137973    -0.168136    1.65218    …  -0.392907   -0.0498037
 -1.93143    -0.431774    -0.729711    0.57885        0.0891769  -2.31571
 -0.275619   -1.97767      0.344649   -0.257833       1.82559     0.581243
 -0.132816    0.0393465    3.82573    -0.6159         0.187089    0.605982
  0.636026    0.215633    -1.53364     1.52166       -0.043997    0.803863
 -0.454117    1.24044      0.212226    0.390823   …   0.930079   -0.00264196
 -0.404232    0.400938     0.180427    0.180604      -0.0598841  -0.015303
  0.104326   -0.0533569   -0.310413    0.1665         0.368739    0.180631
 

### Conditional sampling

In [105]:
Nx = 100
Ny = 50
m = 20
Ne = 500
Xprior = randn(Nx, Ne).*randn(Nx, Ne);
Xpost = deepcopy(Xprior) .+ 0.2*randn(Nx, Ne);

In [106]:
C = MapComponent(m, 5)

MapComponent(20, 1, 5, IntegratedFunction(20, 1, 5, Rectifier("softplus"), ParametricFunction(ExpandedFunction(20, 1, 5, MultiBasis(Basis of 20 functions: Constant -> 18th degree Probabilistic Hermite function
, 5), [0 0 … 0 0], [0.0]))), 1.0e-6)

In [107]:
M = HermiteMap(m, Xprior; diag = true);

In [108]:
Ystar = deepcopy(Xpost[1:Ny,:])# + 0.05*randn(Ny,Ne);

In [109]:
@time M = HermiteMap(m, Xprior);
@time optimize(M, Xprior, 5; withconstant = false, start = Ny+1);

  0.000124 seconds (140 allocations: 85.875 KiB)
  5.008183 seconds (28.98 M allocations: 7.403 GiB, 6.55% gc time)


In [110]:
F = evaluate(M, Xpost; start = Ny+1)

100×500 Array{Float64,2}:
  0.0         0.0         0.0        …   0.0         0.0         0.0
  0.0         0.0         0.0            0.0         0.0         0.0
  0.0         0.0         0.0            0.0         0.0         0.0
  0.0         0.0         0.0            0.0         0.0         0.0
  0.0         0.0         0.0            0.0         0.0         0.0
  0.0         0.0         0.0        …   0.0         0.0         0.0
  0.0         0.0         0.0            0.0         0.0         0.0
  0.0         0.0         0.0            0.0         0.0         0.0
  0.0         0.0         0.0            0.0         0.0         0.0
  0.0         0.0         0.0            0.0         0.0         0.0
  0.0         0.0         0.0        …   0.0         0.0         0.0
  0.0         0.0         0.0            0.0         0.0         0.0
  0.0         0.0         0.0            0.0         0.0         0.0
  ⋮                                  ⋱                          
 -0.917402  

In [112]:
@time inverse!(F, M, Xprior, Ystar; start = Ny+1, P = serial)

  0.019538 seconds (18.26 k allocations: 53.562 MiB)


100×500 Array{Float64,2}:
  0.242942   -0.674307   -2.74656    …   0.804218   -0.322983   -0.737133
 -1.70831     0.873537    2.29053       -0.220672   -0.0823247  -0.225677
  0.298471    1.1673      0.700805      -0.160532    0.610503   -0.0360396
  0.614658   -0.204075    0.0455509      0.145654   -0.0622255  -0.233393
  0.546884   -0.318872   -0.637745      -0.331167    0.0397091   0.189973
 -0.0760382   0.56365    -1.26322    …  -0.125108   -0.360726    0.195504
  1.01948     1.07219    -0.179903      -0.488727   -1.23702     0.0724941
 -0.358755    0.0883633   1.16126       -0.652528   -0.208091    1.48382
 -0.786095    0.0936246   0.417011       0.0718973   1.54053    -0.58896
  0.177832    0.0564213  -0.496262       1.01934    -1.48383    -1.80039
 -0.85691     0.268713   -0.074365   …   0.329593   -1.08626     0.757449
  0.129498   -0.694808    0.142626       0.0568646  -0.351805    0.211033
  0.244564   -0.150814    0.379639      -1.54175    -0.706603    0.254943
  ⋮          

In [115]:
@time inverse!(F, M, Xprior, Ystar; start = Ny+1, P = thread)

  0.000001 seconds (1 allocation: 208 bytes)
  0.036494 seconds (137.49 k allocations: 60.790 MiB)


100×500 Array{Float64,2}:
  0.242942   -0.674307   -2.74656    …   0.804218   -0.322983   -0.737133
 -1.70831     0.873537    2.29053       -0.220672   -0.0823247  -0.225677
  0.298471    1.1673      0.700805      -0.160532    0.610503   -0.0360396
  0.614658   -0.204075    0.0455509      0.145654   -0.0622255  -0.233393
  0.546884   -0.318872   -0.637745      -0.331167    0.0397091   0.189973
 -0.0760382   0.56365    -1.26322    …  -0.125108   -0.360726    0.195504
  1.01948     1.07219    -0.179903      -0.488727   -1.23702     0.0724941
 -0.358755    0.0883633   1.16126       -0.652528   -0.208091    1.48382
 -0.786095    0.0936246   0.417011       0.0718973   1.54053    -0.58896
  0.177832    0.0564213  -0.496262       1.01934    -1.48383    -1.80039
 -0.85691     0.268713   -0.074365   …   0.329593   -1.08626     0.757449
  0.129498   -0.694808    0.142626       0.0568646  -0.351805    0.211033
  0.244564   -0.150814    0.379639      -1.54175    -0.706603    0.254943
  ⋮          

In [101]:
evaluate(M, Xpost)

100×500 Array{Float64,2}:
  0.0120299  -0.457793   -0.814992    …  -0.886963    -0.493031    3.06118
 -0.48832     0.199365   -0.28381         0.00341344  -0.310902   -2.90599
  0.312277    0.432184    0.514434        0.673719    -0.437795   -0.489876
  0.697451    0.187943    0.81117         0.034535     1.36286     0.612952
  0.815565   -0.783322   -1.3412          0.336289     0.135055    1.33254
  0.0498835   0.298793   -0.83403     …  -0.123133     0.210321    0.700186
  0.332829   -0.526824    0.915144        0.814047    -1.53397    -0.409846
 -0.347906   -0.166963   -0.461278        1.99173      0.0767384  -1.73049
 -0.215118   -0.70569    -0.101395       -1.079        1.612      -0.0838957
  0.463558    0.398111    0.946535        1.32585     -0.171809    0.182499
  0.878665   -0.626965    0.0375154   …  -0.0506685    1.83219     0.116359
  1.25871     1.14095    -0.555502       -1.01995     -2.73047    -0.207843
 -0.550401    0.628096   -0.234428        0.553723     0.411383  

In [102]:
100*norm(evaluate(M, Xprior; start = Ny+1)-evaluate(M, Xpost; start = Ny+1))/norm(evaluate(M, Xpost; start = Ny+1))

1.6026660813971276e-14

In [103]:
100*norm(Xprior[Ny+1:end,:] - Xpost[Ny+1:end,:])/norm(Xpost[Ny+1:end,:])

2.130522789901884e-14

In [104]:
Threads.nthreads()

8

In [151]:
?Iterators.partition

```
partition(collection, n)
```

Iterate over a collection `n` elements at a time.

# Examples

```jldoctest
julia> collect(Iterators.partition([1,2,3,4,5], 2))
3-element Array{SubArray{Int64,1,Array{Int64,1},Tuple{UnitRange{Int64}},true},1}:
 [1, 2]
 [3, 4]
 [5]
```


In [149]:
Iterators.partition(1:15,2)

Base.Iterators.PartitionIterator{UnitRange{Int64}}(1:15, 2)

In [150]:
parititionPoints = range(1, stop=13, count=5)

MethodError: MethodError: no method matching range(::Int64; stop=13, count=5)
Closest candidates are:
  range(::Any; length, stop, step) at range.jl:91 got unsupported keyword argument "count"
  range(::Any, !Matched::Any; length, step) at range.jl:94 got unsupported keyword arguments "stop", "count"
  range(!Matched::LaurentPolynomial) at /home/mat/.julia/packages/Polynomials/ZmARV/src/polynomials/LaurentPolynomial.jl:144 got unsupported keyword arguments "stop", "count"
  ...